### Glacier Ice and Snow Calculator.

Steps for the initial part of the project:-
1 - Gather Satellite images of a glacier mountainscape (Kenai Mountains) - Using Landsat images
2 - Present 3 different time periods (1999, 2010 - 2025 using landsat)
3 - Covert images into false colour composite
4 - Conduct a classification on both landsat scenes
5 - calculate area coverage (km2)
6 - present results


Potential expansions:-
- Expand the study period and get more mountains glaciers
- Compare volume changes

In [149]:
import os
import geopandas as gpd
import rasterio as rio
import rasterio.merge
import shapely
import usgs

In [167]:
#As Landsat imagery is going to be the source of satellite imagery covering the glaciers used in this study, the first step is to log in to the USGS API using a username and token
from usgs import api

# Open and manually read the username from the .usgs_user file in the home directory
with open('C:/Users/couse/.usgs_user', 'r') as usertext:
    username = usertext.read()

#Open and manually read the password/token from the.usgs_token in the home directory
with open('C:/Users/couse/.usgs_token', 'r') as tokentext:
    password = tokentext.read()
    
# Login to the API using the username and the token(API_KEY provided by the USGS) as password
login = api.login(username, password, save=True)

#Below the API Key (token) is extracted separately as it will be used for the functions later in the script
api_key = login['data']

In [77]:
# To get a better understanding of all the functions and methods available in the 'usgs.api' a directory is printed below
print(dir(usgs.api))

['FuturesSession', 'HTTPAdapter', 'Retry', 'TMPFILE', 'USGSAuthExpiredError', 'USGSError', 'USGS_API', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__version__', '_check_for_usgs_error', '_create_session', '_get_api_key', 'dataset_download_options', 'dataset_filters', 'dataset_search', 'datetime', 'download_options', 'download_request', 'json', 'login', 'logout', 'os', 'payloads', 'requests', 'scene_metadata', 'scene_search']


In [113]:
help(api.dataset_search)

Help on function dataset_search in module usgs.api:

dataset_search(dataset=None, catalog=None, ll=None, ur=None, start_date=None, end_date=None, api_key=None)



In [ ]:
Available_Satellites = api.dataset_search()
print(Available_Satellites)

In [163]:
# The Landsat images comes from the EarthExplorer (EE) dataset (node), in order to get an idea of what Landsat data is available
satellite_avail = api.dataset_search("landsat", "EE", api_key=api_key)

# The code below prints out each Satellites dataset identified in the api.dataset_search which are available for this project using the provided API Key
for dataset in satellite_avail['data']:
        print(dataset['datasetAlias'])

esat_etm_nopan
esat_etm_pan
ortho_mosaic_etm
geos_5_fp_it
geos_5_it
lima
landsat_mss_c2_l1
landsat_tm_c2_l1
landsat_tm_c2_l2
landsat_ba_tile_c2
landsat_dswe_tile_c2
landsat_fsca_tile_c2
landsat_fsca_tile_stat_c2
landsat_ard_tile_c2
landsat_etm_c2_l1
landsat_etm_c2_l2
landsat_ot_c2_l1
landsat_ot_c2_l2
landsat_band_files_c2_l1
landsat_band_files_c2_l2
lima_mosaic
merra_2_c2
ortho_mss_scene
mss_film
rbv_film
sys_etm
esat_tm
tm_film
ortho_mosaic
viirs_atmos


In [62]:
#First we are going to prepare the dataset by creating a Bounding Box and selecting the dates for the images we are going to use from earthaccess

KenaiMountains_BBox = (-161.5, 59.5, -158.0, 60.8)
years = ["1999-08-01", "2010-08-01", "2025-08-01"] #Summer months have been selected so that we can have clearer vision of glacier boundaries

#We will use the following satellites to acquire images over the Kenai Mountains for each respective time period
datasets = {
    "2000": "LANDSAT_7_C02_T1",  
    "2011": "LANDSAT_7_C02_T1",
    "2025": "LANDSAT_9_C2_L1" 
}

In [ ]:
# Get filters for the dataset and node
filters = api.dataset_filters('landsat_ot_c2_l1', api_key)

# Show filter options
import json
print(json.dumps(filters, indent=2))

In [106]:
where = {
    "filter_id": '5e81f14f8d2a7c24',
    "value": "043",
    "operand": "="
}


In [ ]:
# Perform the scene search with filters
results = api.scene_search(
    dataset="landsat_ot_c2_l1",  # Specify the dataset (e.g., LANDSAT_8_C1)
    start_date="2017-04-01",  # Start date for the scene search
    end_date="2017-05-01",    # End date for the scene search
    where=where,              # Apply the 'where' filter
    max_results=10,           # Limit the number of results
    api_key=api_key    # Provide your API key here
)
for scene in results['data']['results']:
    print(scene['entityId'])  # or 'entity_id' depending on API naming



print(len(results))

In [10]:
help(api.scene_search)


Help on function scene_search in module usgs.api:

scene_search(dataset, max_results=5000, metadata_type=None, start_date=None, end_date=None, ll=None, ur=None, lat=None, lng=None, distance=100, where=None, starting_number=1, sort_order='DESC', api_key=None)
    :param dataset:
        USGS dataset (e.g. EO1_HYP_PUB, LANDSAT_8)
    :param lat:
        Latitude
    :param lng:
        Longitude
    :param distance:
        Distance in meters used to for a radial search
    :param ll:
        Dictionary of longitude/latitude coordinates for the lower left corner
        of a bounding box search. e.g. { "longitude": 0.0, "latitude": 0.0 }
    :param ur:
        Dictionary of longitude/latitude coordinates for the upper right corner
        of a bounding box search. e.g. { "longitude": 0.0, "latitude": 0.0 }
    :param start_date:
        Start date for when a scene has been acquired
    :param end_date:
        End date for when a scene has been acquired
    :where:
        Dictionary rep

In [148]:
help(api.logout)

Help on function logout in module usgs.api:

logout()
    Log out by deactivating and removing the stored API key, if one exists.



In [166]:
#Logout and Deactivate the API Key used in the login
api.logout()

{'errorCode': 'AUTH_EXPIRED',
 'errorMessage': 'API key has expired due to inactivity, please logout and re-login.',
 'data': None,
 'requestId': 0,
 'sessionId': None,
 'version': None}

In [170]:
help(api.dataset_download_options)

Help on function dataset_download_options in module usgs.api:

dataset_download_options(dataset, api_key=None)
    The dataset download options method is used to discover downloadable
    products for a specified dataset. Unlike the `download_options` method,
    this does not check product availability.
    
    :param str dataset: Used to identify the which dataset to return results for.



In [171]:
api.dataset_download_options(
    dataset = "landsat_ot_c2_l1",
    api_key = api_key)

{'requestId': 1981786291,
 'version': 'stable',
 'sessionId': 314900882,
 'data': [{'productId': '632211e26883b1f7',
   'productCode': 'D805',
   'productName': 'Landsat Collection 2 Level-1 Product Bundle',
   'downloadName': '',
   'downloadSystem': 'dds_ms',
   'fileGroups': None},
  {'productId': '73ceb05468b7e8c2',
   'productCode': 'D687',
   'productName': 'Landsat Collection 2 Level-1 Band File',
   'downloadName': 'C2L1 Tile Product Files',
   'downloadSystem': 'folder',
   'fileGroups': None},
  {'productId': '6447cc2e2a09aab6',
   'productCode': 'D689',
   'productName': 'Landsat Collection 2 Level-1 Band File',
   'downloadName': 'C2L1 Tile Product Files',
   'downloadSystem': 'folder',
   'fileGroups': None},
  {'productId': '5e81f14f92acf9ef',
   'productCode': 'D690',
   'productName': 'Landsat Collection 2 Level-1 Product Bundle',
   'downloadName': '',
   'downloadSystem': 'ls_zip',
   'fileGroups': None},
  {'productId': '5e9eb01274d7924f',
   'productCode': 'D734',
 

In [172]:
help(api.download_request)

Help on function download_request in module usgs.api:

download_request(dataset, entity_id, product_id, api_key=None)
    This method is used to insert the requested downloads into the download queue
    and returns the available download URLs.



In [173]:
help(api.download_options)

Help on function download_options in module usgs.api:

download_options(dataset, entity_ids, api_key=None)



In [ ]:
api.download_options(
    dataset="landsat_ot_c2_l1",
    entity_ids="LC81380432017120LGN00",
    api_key=api_key)

In [191]:
download_options = api.download_request(
    dataset="landsat_ot_c2_l1",
    entity_id=[
        "LC80720432017121LGN00",
        "LC81290432017121LGN00",
        "LC81450432017121LGN00"
    ],
    produc
    api_key=api_key
)
print(download_options)

TypeError: download_request() missing 1 required positional argument: 'product_id'

In [192]:
help(api.download_request)

Help on function download_request in module usgs.api:

download_request(dataset, entity_id, product_id, api_key=None)
    This method is used to insert the requested downloads into the download queue
    and returns the available download URLs.

